1. Konfigurasi API dan Inisialisasi Klien
Ganti nilai CDS_API_KEY dengan API key Anda.

2. Fungsi Download Data ERA5 (Mengambil Data Setahun Penuh)  
Request ini meminta semua bulan dan semua hari pada tahun yang ditentukan di Cell 2.

3. Pemrosesan Data (Agregasi Harian & Konversi Unit)  
Ganti YEAR_TO_PROCESS dengan tahun yang baru selesai Anda unduh.

2025-10-31 22:06:11,706 - INFO - 🚀 MEMULAI ERA5 DATA PROCESSING PIPELINE
2025-10-31 22:06:11,708 - INFO - 🔧 Setup environment...
2025-10-31 22:06:11,709 - INFO - ✅ xarray tersedia
2025-10-31 22:06:11,710 - INFO - ✅ pandas tersedia
2025-10-31 22:06:11,711 - INFO - ✅ numpy tersedia
2025-10-31 22:06:11,713 - INFO - ✅ cdsapi tersedia
2025-10-31 22:06:11,716 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\raw_data


2025-10-31 22:06:11,719 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\processed_data
2025-10-31 22:06:11,722 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\csv_output
2025-10-31 22:06:11,724 - INFO - 📥 Download data ERA5...
2025-10-31 22:06:23,017 - WARNING - Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/catalogue/v1/messages (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001A64C27EE90>: Failed to resolve 'cds.climate.copernicus.eu' ([Errno 11001] getaddrinfo failed)"))], attempt 1 of 500
2025-10-31 22:06:23,020 - WARNING - Retrying in 120 seconds
2025-10-31 22:08:23,023 - INFO - Retrying now...
2025-10-31 22:08:30,058 - WARNING - Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retrie

2025-11-06 10:44:39,152 - INFO - 🚀 MEMULAI ERA5 DATA PROCESSING PIPELINE
2025-11-06 10:44:39,155 - INFO - 🔧 Setup environment...
2025-11-06 10:44:39,156 - INFO - ✅ xarray tersedia
2025-11-06 10:44:39,157 - INFO - ✅ pandas tersedia
2025-11-06 10:44:39,157 - INFO - ✅ numpy tersedia
2025-11-06 10:44:39,159 - INFO - ✅ cdsapi tersedia
2025-11-06 10:44:39,906 - INFO - ✅ netcdf4 tersedia
2025-11-06 10:44:39,911 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\raw_data
2025-11-06 10:44:39,915 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\processed_data
2025-11-06 10:44:39,918 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\csv_output
2025-11-06 10:44:39,922 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\temp_extract
2025-11-06 10:44:39,925 - INFO -

In [4]:
import cdsapi
import xarray as xr
import numpy as np
import pandas as pd
import os
import glob
import warnings
import zipfile
import shutil
from itertools import product
from pathlib import Path
import logging
from datetime import datetime, timedelta

# Setup logging untuk monitoring yang lebih baik
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Suppress warnings yang tidak penting
warnings.filterwarnings('ignore')

# =============================================================================
# KONFIGURASI
# =============================================================================

class ERA5Config:
    """Konfigurasi untuk download dan processing data ERA5"""
    
    # API Configuration
    CDS_API_KEY = "0454455d-cef6-4a4e-ad46-f0e905629467" 
    CDS_API_URL = "https://cds.climate.copernicus.eu/api"
    
    # Target years dan months
    TARGET_YEARS = ['2023']
    TARGET_MONTHS = ['01','02','03' '04', '05', '06', '07', '08', '09', '10', '11', '12']  # February dan March
    
    # Lokasi spesifik yang ingin diambil datanya
    LOCATIONS = {
        'bundaran_hi': {'lat': -6.19466, 'lon': 106.8235},
        'kelapa_gading': {'lat': -6.15358, 'lon': 106.91089},
        'jagakarsa': {'lat': -6.35693, 'lon': 106.80367},
        'lubang_buaya': {'lat': -6.28889, 'lon': 106.90919},
        'kebun_jeruk': {'lat': -6.20735, 'lon': 106.75319}
    }
    
    # Area bounds untuk download (sedikit lebih besar dari area lokasi)
    AREA_BOUNDS = [
        max(loc['lat'] for loc in LOCATIONS.values()) + 0.1,  # North
        min(loc['lon'] for loc in LOCATIONS.values()) - 0.1,  # West  
        min(loc['lat'] for loc in LOCATIONS.values()) - 0.1,  # South
        max(loc['lon'] for loc in LOCATIONS.values()) + 0.1   # East
    ]
    
    # Variables yang akan didownload
    VARIABLES = [
        '10m_u_component_of_wind', '10m_v_component_of_wind', 
        '2m_dewpoint_temperature', '2m_temperature', 
        'total_precipitation'
    ]
    
    # Path configuration
    BASE_PATH = 'C:\\Users\\user\\OneDrive\\IPB\\Thesis\\02. Development\\02. Data ERA5\\01. Data Praprocessing'
    RAW_DATA_PATH = os.path.join(BASE_PATH, 'raw_data')
    PROCESSED_DATA_PATH = os.path.join(BASE_PATH, 'processed_data')
    CSV_OUTPUT_PATH = os.path.join(BASE_PATH, 'csv_output')
    TEMP_EXTRACT_PATH = os.path.join(BASE_PATH, 'temp_extract')

# =============================================================================
# FUNGSI UTILITY
# =============================================================================

def setup_directories():
    """Membuat direktori yang diperlukan"""
    paths = [
        ERA5Config.RAW_DATA_PATH,
        ERA5Config.PROCESSED_DATA_PATH, 
        ERA5Config.CSV_OUTPUT_PATH,
        ERA5Config.TEMP_EXTRACT_PATH
    ]
    
    for path in paths:
        os.makedirs(path, exist_ok=True)
        logger.info(f"Directory created/verified: {path}")

def check_dependencies():
    """Memeriksa semua dependencies yang diperlukan"""
    required_packages = {
        'xarray': 'xarray',
        'pandas': 'pandas', 
        'numpy': 'numpy',
        'cdsapi': 'cdsapi',
        'netcdf4': 'netCDF4'
    }
    
    missing_packages = []
    for package, import_name in required_packages.items():
        try:
            __import__(import_name)
            logger.info(f"✅ {package} tersedia")
        except ImportError:
            missing_packages.append(package)
            logger.error(f"❌ {package} tidak tersedia")
    
    if missing_packages:
        logger.error(f"Package yang missing: {missing_packages}")
        logger.error("Install dengan: pip install " + " ".join(missing_packages))
        return False
    
    return True

def extract_zip_file(zip_path, extract_to=None):
    """
    Extract file ZIP dan cari file NetCDF di dalamnya
    """
    if extract_to is None:
        extract_to = ERA5Config.TEMP_EXTRACT_PATH
    
    try:
        logger.info(f"Mengekstrak file ZIP: {zip_path}")
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Dapatkan list file dalam ZIP
            file_list = zip_ref.namelist()
            logger.info(f"File dalam ZIP: {file_list}")
            
            # Extract semua file
            zip_ref.extractall(extract_to)
            
            # Cari file NetCDF
            nc_files = [f for f in file_list if f.endswith('.nc')]
            if nc_files:
                nc_file_path = os.path.join(extract_to, nc_files[0])
                logger.info(f"File NetCDF ditemukan: {nc_file_path}")
                return nc_file_path
            else:
                logger.warning("Tidak ada file NetCDF dalam ZIP")
                return None
                
    except Exception as e:
        logger.error(f"Error extracting ZIP file: {str(e)}")
        return None

def handle_zip_download(downloaded_file, target_nc_file):
    """
    Handle file yang didownload sebagai ZIP tetapi disimpan sebagai .nc
    """
    try:
        # Cek jika file sebenarnya adalah ZIP
        if zipfile.is_zipfile(downloaded_file):
            logger.info(f"File {downloaded_file} adalah file ZIP")
            
            # Extract ZIP file
            extracted_nc = extract_zip_file(downloaded_file)
            
            if extracted_nc and os.path.exists(extracted_nc):
                # Copy file yang diekstrak ke lokasi target
                shutil.copy2(extracted_nc, target_nc_file)
                logger.info(f"File NetCDF disimpan sebagai: {target_nc_file}")
                
                # Hapus file ZIP asli
                os.remove(downloaded_file)
                logger.info(f"File ZIP asli dihapus: {downloaded_file}")
                
                return True
            else:
                logger.error("Gagal mengekstrak file NetCDF dari ZIP")
                return False
        else:
            logger.info(f"File {downloaded_file} adalah file NetCDF asli")
            return True
            
    except Exception as e:
        logger.error(f"Error handling ZIP download: {str(e)}")
        return False

# =============================================================================
# DOWNLOAD DATA ERA5
# =============================================================================

class ERA5Downloader:
    """Class untuk handle download data ERA5"""
    
    def __init__(self):
        self.client = cdsapi.Client(
            url=ERA5Config.CDS_API_URL, 
            key=ERA5Config.CDS_API_KEY
        )
        logger.info("CDS API client initialized")
    
    def download_monthly_data(self, year, month):
        """Download data bulanan untuk tahun dan bulan tertentu"""
        
        output_file = os.path.join(
            ERA5Config.RAW_DATA_PATH, 
            f'data_era5_hourly_{year}_{month}.nc'
        )
        
        # Skip jika file sudah ada dan ukurannya reasonable
        if (os.path.exists(output_file) and 
            os.path.getsize(output_file) > 10_000_000):  # > 10MB
            logger.info(f"File {output_file} sudah ada, skip download")
            return output_file
        
        # Prepare request parameters
        all_days = [f'{i:02d}' for i in range(1, 32)]
        all_hours = [f'{h:02d}:00' for h in range(24)]
        
        request_params = {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': ERA5Config.VARIABLES,
            'year': year,
            'month': month,
            'day': all_days,
            'time': all_hours,
            'area': ERA5Config.AREA_BOUNDS,
        }
        
        try:
            logger.info(f"Downloading data untuk {year}-{month}...")
            
            # Download ke temporary file dulu
            temp_file = output_file + '.temp'
            self.client.retrieve(
                'reanalysis-era5-single-levels',
                request_params,
                temp_file
            )
            
            # Handle kemungkinan file ZIP
            if handle_zip_download(temp_file, output_file):
                logger.info(f"✅ Download dan ekstrak selesai: {output_file}")
                return output_file
            else:
                logger.error(f"❌ Gagal processing file untuk {year}-{month}")
                return None
            
        except Exception as e:
            logger.error(f"❌ Gagal download {year}-{month}: {str(e)}")
            # Hapus file temporary jika ada
            if os.path.exists(temp_file):
                os.remove(temp_file)
            return None
    
    def download_all_data(self):
        """Download semua data berdasarkan konfigurasi"""
        downloaded_files = []
        
        for year, month in product(ERA5Config.TARGET_YEARS, ERA5Config.TARGET_MONTHS):
            result = self.download_monthly_data(year, month)
            if result:
                downloaded_files.append(result)
        
        logger.info(f"Total files downloaded: {len(downloaded_files)}")
        return downloaded_files

# =============================================================================
# PROCESSING DATA NETCDF KE CSV (PER LOKASI SPESIFIK)
# =============================================================================

class ERA5Processor:
    """Class untuk processing data ERA5 dari NetCDF ke CSV per lokasi spesifik"""
    
    @staticmethod
    def try_open_netcdf(file_path):
        """Mencoba berbagai engine untuk membuka file NetCDF"""
        engines_to_try = ['netcdf4', 'scipy', 'h5netcdf']
        
        for engine in engines_to_try:
            try:
                logger.info(f"Mencoba buka {file_path} dengan engine: {engine}")
                ds = xr.open_dataset(file_path, engine=engine)
                logger.info(f"✅ Berhasil buka file dengan engine: {engine}")
                return ds
            except Exception as e:
                logger.warning(f"❌ Engine {engine} gagal: {str(e)}")
                continue
        
        # Jika semua engine gagal, coba tanpa specify engine
        try:
            logger.info("Mencoba buka file tanpa specify engine...")
            ds = xr.open_dataset(file_path)
            logger.info("✅ Berhasil buka file tanpa specify engine")
            return ds
        except Exception as e:
            logger.error(f"❌ Semua metode gagal: {str(e)}")
            return None

    @staticmethod
    def find_nearest_grid_point(ds, target_lat, target_lon):
        """Mencari titik grid terdekat dengan lokasi target"""
        try:
            # Calculate absolute differences
            lat_diff = np.abs(ds.latitude.values - target_lat)
            lon_diff = np.abs(ds.longitude.values - target_lon)
            
            # Find indices of minimum differences
            lat_idx = np.argmin(lat_diff)
            lon_idx = np.argmin(lon_diff)
            
            # Get actual coordinates
            actual_lat = ds.latitude.values[lat_idx]
            actual_lon = ds.longitude.values[lon_idx]
            
            logger.info(f"Target: ({target_lat:.4f}, {target_lon:.4f}) -> Grid: ({actual_lat:.4f}, {actual_lon:.4f})")
            
            return lat_idx, lon_idx, actual_lat, actual_lon
            
        except Exception as e:
            logger.error(f"Error finding nearest grid point: {str(e)}")
            return None, None, None, None

    @staticmethod
    def extract_data_for_location(ds, location_name, target_lat, target_lon):
        """Extract data untuk satu lokasi spesifik"""
        try:
            # Find nearest grid point
            lat_idx, lon_idx, actual_lat, actual_lon = ERA5Processor.find_nearest_grid_point(
                ds, target_lat, target_lon
            )
            
            if lat_idx is None:
                return None
            
            # Extract data untuk titik tersebut
            location_data = {}
            
            for var_name in ds.data_vars:
                try:
                    # Select data untuk titik lokasi
                    var_data = ds[var_name].isel(latitude=lat_idx, longitude=lon_idx)
                    
                    # Convert to pandas Series
                    var_series = var_data.to_series()
                    
                    # Store in dictionary
                    location_data[var_name] = var_series
                    
                except Exception as e:
                    logger.warning(f"Error extracting {var_name} for {location_name}: {str(e)}")
                    continue
            
            if not location_data:
                logger.error(f"Tidak ada data yang berhasil diekstrak untuk {location_name}")
                return None
            
            # Create DataFrame
            df = pd.DataFrame(location_data)
            df.reset_index(inplace=True)
            
            # Debug: Print column names untuk troubleshooting
            logger.info(f"Columns in DataFrame: {df.columns.tolist()}")
            
            # Cari kolom waktu yang benar
            time_column = None
            possible_time_columns = ['valid_time', 'time', 'datetime', 'index']
            for col in possible_time_columns:
                if col in df.columns:
                    time_column = col
                    break
            
            if time_column is None:
                # Jika tidak ada kolom waktu yang jelas, gunakan index
                df['datetime'] = df.index
                time_column = 'datetime'
            
            # Rename kolom waktu ke 'datetime'
            if time_column != 'datetime':
                df.rename(columns={time_column: 'datetime'}, inplace=True)
            
            # Add location information
            df['location_name'] = location_name
            df['latitude'] = actual_lat
            df['longitude'] = actual_lon
            
            # Rename columns untuk lebih jelas
            column_mapping = {
                't2m': 'temperature_2m',
                'd2m': 'dewpoint_temperature_2m',
                'u10': 'u_wind_10m',
                'v10': 'v_wind_10m',
                'tp': 'precipitation'
            }
            
            df.rename(columns=column_mapping, inplace=True)
            
            # Calculate additional variables
            if 'u_wind_10m' in df.columns and 'v_wind_10m' in df.columns:
                df['wind_speed_10m'] = np.sqrt(df['u_wind_10m']**2 + df['v_wind_10m']**2)
                df['wind_direction_10m'] = np.arctan2(df['v_wind_10m'], df['u_wind_10m']) * 180 / np.pi
                df['wind_direction_10m'] = (df['wind_direction_10m'] + 360) % 360  # Convert to 0-360
            
            # Convert temperature from Kelvin to Celsius
            if 'temperature_2m' in df.columns:
                df['temperature_2m_c'] = df['temperature_2m'] - 273.15
            
            if 'dewpoint_temperature_2m' in df.columns:
                df['dewpoint_temperature_2m_c'] = df['dewpoint_temperature_2m'] - 273.15
            
            logger.info(f"✅ Data untuk {location_name}: {len(df)} records, columns: {df.columns.tolist()}")
            return df
            
        except Exception as e:
            logger.error(f"Error extracting data for {location_name}: {str(e)}")
            return None

    @staticmethod
    def convert_era5_to_csv_per_location(nc_file_path, output_dir=None):
        """
        Konversi file NetCDF ERA5 ke format CSV dengan data per lokasi spesifik
        """
        try:
            logger.info(f"Memproses file: {nc_file_path}")
            
            # Validasi file
            if not os.path.exists(nc_file_path):
                logger.error(f"File tidak ditemukan: {nc_file_path}")
                return None
            
            file_size = os.path.getsize(nc_file_path) / (1024 * 1024)  # MB
            logger.info(f"Ukuran file: {file_size:.2f} MB")
            
            # Cek jika file adalah ZIP
            if zipfile.is_zipfile(nc_file_path):
                logger.info(f"File {nc_file_path} adalah ZIP file, mengekstrak...")
                extracted_file = extract_zip_file(nc_file_path)
                if extracted_file:
                    nc_file_path = extracted_file
                else:
                    logger.error("Gagal mengekstrak file ZIP")
                    return None
            
            # Buka file NetCDF dengan multiple engine fallback
            ds = ERA5Processor.try_open_netcdf(nc_file_path)
            if ds is None:
                return None
            
            # Ekstrak informasi dataset
            logger.info(f"Dimensi dataset: {dict(ds.dims)}")
            logger.info(f"Koordinat dataset: {list(ds.coords)}")
            logger.info(f"Variabel yang tersedia: {list(ds.data_vars)}")
            
            # Extract data untuk setiap lokasi
            all_location_data = []
            
            for location_name, coords in ERA5Config.LOCATIONS.items():
                logger.info(f"📍 Memproses lokasi: {location_name}")
                
                location_df = ERA5Processor.extract_data_for_location(
                    ds, location_name, coords['lat'], coords['lon']
                )
                
                if location_df is not None:
                    all_location_data.append(location_df)
            
            if not all_location_data:
                logger.error("Tidak ada data yang berhasil diekstrak untuk semua lokasi")
                ds.close()
                return None
            
            # Gabungkan data semua lokasi
            combined_df = pd.concat(all_location_data, ignore_index=True)
            
            # Optimasi tipe data
            combined_df = ERA5Processor.optimize_dataframe(combined_df)
            
            # Urutkan data
            if 'datetime' in combined_df.columns:
                combined_df.sort_values(['datetime', 'location_name'], inplace=True)
            
            # Simpan ke CSV
            base_name = Path(nc_file_path).stem
            if output_dir is None:
                output_dir = ERA5Config.CSV_OUTPUT_PATH
            
            output_file = os.path.join(output_dir, f"{base_name}_per_location.csv")
            
            # Simpan ke CSV
            combined_df.to_csv(output_file, index=False)
            logger.info(f"✅ File CSV disimpan: {output_file}")
            
            # Log statistics
            ERA5Processor.log_data_statistics(combined_df)
            
            ds.close()
            return output_file
            
        except Exception as e:
            logger.error(f"❌ Error processing {nc_file_path}: {str(e)}")
            import traceback
            logger.error(f"Traceback: {traceback.format_exc()}")
            return None

    @staticmethod
    def optimize_dataframe(df):
        """Optimasi tipe data untuk menghemat memory"""
        # Optimasi numeric columns
        float_cols = df.select_dtypes(include=['float64']).columns
        for col in float_cols:
            if col in df.columns:
                df[col] = df[col].astype(np.float32)
        
        return df
    
    @staticmethod
    def log_data_statistics(df):
        """Log statistics data"""
        logger.info("📊 Data Statistics:")
        logger.info(f"   Total rows: {len(df):,}")
        logger.info(f"   Total columns: {len(df.columns)}")
        
        if 'datetime' in df.columns:
            logger.info(f"   Time range: {df['datetime'].min()} to {df['datetime'].max()}")
        
        logger.info(f"   Locations: {df['location_name'].unique().tolist()}")
        logger.info(f"   Available variables: {[col for col in df.columns if col not in ['datetime', 'location_name', 'latitude', 'longitude']]}")
        
        # Preview data
        logger.info("👀 Preview data (first 5 rows):")
        print(df.head().to_string())

    @staticmethod
    def batch_convert_era5_files(input_pattern=None, output_dir=None):
        """Konversi batch semua file ERA5"""
        
        if input_pattern is None:
            input_pattern = os.path.join(ERA5Config.RAW_DATA_PATH, "data_era5_hourly_*.nc")
        
        if output_dir is None:
            output_dir = ERA5Config.CSV_OUTPUT_PATH
        
        # Cari file NetCDF
        nc_files = glob.glob(input_pattern)
        
        if not nc_files:
            logger.warning(f"Tidak ada file ditemukan dengan pattern: {input_pattern}")
            
            # Coba pattern alternatif
            alternative_patterns = [
                os.path.join(ERA5Config.RAW_DATA_PATH, "*.nc"),
                "data_era5_hourly_*.nc",
                "era5_*.nc"
            ]
            
            for pattern in alternative_patterns:
                nc_files = glob.glob(pattern)
                if nc_files:
                    logger.info(f"Menggunakan pattern alternatif: {pattern}")
                    break
        
        if not nc_files:
            logger.error("Tidak ada file NetCDF yang ditemukan!")
            return []
        
        logger.info(f"Menemukan {len(nc_files)} file untuk dikonversi:")
        for i, nc_file in enumerate(sorted(nc_files), 1):
            logger.info(f"  {i:2d}. {nc_file}")
        
        # Process setiap file
        converted_files = []
        failed_files = []
        
        for nc_file in sorted(nc_files):
            result = ERA5Processor.convert_era5_to_csv_per_location(nc_file, output_dir)
            if result:
                converted_files.append(result)
            else:
                failed_files.append(nc_file)
        
        # Summary
        logger.info(f"\n{'='*50}")
        logger.info("SUMMARY KONVERSI:")
        logger.info(f"{'='*50}")
        logger.info(f"✅ Berhasil: {len(converted_files)} file")
        logger.info(f"❌ Gagal: {len(failed_files)} file")
        
        if failed_files:
            logger.info("File yang gagal:")
            for ff in failed_files:
                logger.info(f"  - {ff}")
        
        return converted_files

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Main function untuk menjalankan seluruh pipeline"""
    logger.info("🚀 MEMULAI ERA5 DATA PROCESSING PIPELINE")
    
    # Step 1: Setup environment
    logger.info("🔧 Setup environment...")
    if not check_dependencies():
        logger.error("Dependencies tidak lengkap, proses dihentikan")
        return
    
    setup_directories()
    
    # Step 2: Download data
    logger.info("📥 Download data ERA5...")
    downloader = ERA5Downloader()
    downloaded_files = downloader.download_all_data()
    
    if not downloaded_files:
        logger.warning("Tidak ada file yang didownload, lanjut ke file yang sudah ada")
    
    # Step 3: Process data ke CSV per lokasi
    logger.info("🔄 Konversi NetCDF ke CSV (per lokasi)...")
    processor = ERA5Processor()
    converted_files = processor.batch_convert_era5_files()
    
    # Final summary
    logger.info(f"\n🎉 PIPELINE SELESAI!")
    logger.info(f"📁 File CSV tersimpan di: {ERA5Config.CSV_OUTPUT_PATH}")
    
    if converted_files:
        logger.info("File yang berhasil dikonversi:")
        for cf in converted_files:
            logger.info(f"  📄 {cf}")
    else:
        logger.error("❌ Tidak ada file yang berhasil dikonversi!")

if __name__ == "__main__":
    main()

2025-11-06 11:54:35,242 - INFO - 🚀 MEMULAI ERA5 DATA PROCESSING PIPELINE
2025-11-06 11:54:35,248 - INFO - 🔧 Setup environment...
2025-11-06 11:54:35,249 - INFO - ✅ xarray tersedia
2025-11-06 11:54:35,250 - INFO - ✅ pandas tersedia
2025-11-06 11:54:35,251 - INFO - ✅ numpy tersedia
2025-11-06 11:54:35,252 - INFO - ✅ cdsapi tersedia
2025-11-06 11:54:35,253 - INFO - ✅ netcdf4 tersedia
2025-11-06 11:54:35,258 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\02. Data ERA5\01. Data Praprocessing\raw_data
2025-11-06 11:54:35,261 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\02. Data ERA5\01. Data Praprocessing\processed_data
2025-11-06 11:54:35,263 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\02. Data ERA5\01. Data Praprocessing\csv_output
2025-11-06 11:54:35,265 - INFO - Directory created/verified: C:\Users\user\OneDrive\IPB\Thesis\02. Development\02. Data ERA5\01. Data Pra

       datetime  u_wind_10m  v_wind_10m  dewpoint_temperature_2m  temperature_2m  location_name  latitude   longitude  wind_speed_10m  wind_direction_10m  temperature_2m_c  dewpoint_temperature_2m_c
0    2023-01-01    3.075758    0.363742               296.163696      297.533478    bundaran_hi    -6.206  106.903999        3.097192            6.744507         24.383484                  23.013702
1488 2023-01-01    1.991102   -0.163923               295.289459      296.147369      jagakarsa    -6.457  106.903999        1.997838          355.293579         22.997375                  22.139465
2976 2023-01-01    2.074171    0.687701               296.270691      297.626465    kebun_jeruk    -6.206  106.653000        2.185204           18.343170         24.476471                  23.120697
744  2023-01-01    3.075758    0.363742               296.163696      297.533478  kelapa_gading    -6.206  106.903999        3.097192            6.744507         24.383484                  23.013702
2232 

2025-11-06 23:19:27,286 - INFO - Columns in DataFrame: ['valid_time', 'u10', 'v10', 'd2m', 't2m']
2025-11-06 23:19:27,298 - INFO - ✅ Data untuk kebun_jeruk: 672 records, columns: ['datetime', 'u_wind_10m', 'v_wind_10m', 'dewpoint_temperature_2m', 'temperature_2m', 'location_name', 'latitude', 'longitude', 'wind_speed_10m', 'wind_direction_10m', 'temperature_2m_c', 'dewpoint_temperature_2m_c']
2025-11-06 23:19:27,423 - INFO - ✅ File CSV disimpan: C:\Users\user\OneDrive\IPB\Thesis\02. Development\02. Data ERA5\01. Data Praprocessing\csv_output\data_era5_hourly_2023_02_per_location.csv
2025-11-06 23:19:27,425 - INFO - 📊 Data Statistics:
2025-11-06 23:19:27,426 - INFO -    Total rows: 3,360
2025-11-06 23:19:27,428 - INFO -    Total columns: 12
2025-11-06 23:19:27,429 - INFO -    Time range: 2023-02-01 00:00:00 to 2023-02-28 23:00:00
2025-11-06 23:19:27,431 - INFO -    Locations: ['bundaran_hi', 'jagakarsa', 'kebun_jeruk', 'kelapa_gading', 'lubang_buaya']
2025-11-06 23:19:27,432 - INFO -   

       datetime  u_wind_10m  v_wind_10m  dewpoint_temperature_2m  temperature_2m  location_name  latitude   longitude  wind_speed_10m  wind_direction_10m  temperature_2m_c  dewpoint_temperature_2m_c
0    2023-02-01    2.091321    0.208390               296.850830      298.195862    bundaran_hi    -6.206  106.903999        2.101677            5.690460         25.045868                  23.700836
1344 2023-02-01    1.059491    0.646531               295.754852      296.526428      jagakarsa    -6.457  106.903999        1.241178           31.392700         23.376434                  22.604858
2688 2023-02-01    1.012463    0.550752               296.784637      298.345642    kebun_jeruk    -6.206  106.653000        1.152566           28.544952         25.195648                  23.634644
672  2023-02-01    2.091321    0.208390               296.850830      298.195862  kelapa_gading    -6.206  106.903999        2.101677            5.690460         25.045868                  23.700836
2016 

2025-11-06 23:19:27,657 - INFO - ✅ Data untuk lubang_buaya: 744 records, columns: ['datetime', 'u_wind_10m', 'v_wind_10m', 'dewpoint_temperature_2m', 'temperature_2m', 'location_name', 'latitude', 'longitude', 'wind_speed_10m', 'wind_direction_10m', 'temperature_2m_c', 'dewpoint_temperature_2m_c']
2025-11-06 23:19:27,660 - INFO - 📍 Memproses lokasi: kebun_jeruk
2025-11-06 23:19:27,666 - INFO - Target: (-6.2073, 106.7532) -> Grid: (-6.2060, 106.6530)
2025-11-06 23:19:27,689 - INFO - Columns in DataFrame: ['valid_time', 'u10', 'v10', 'd2m', 't2m']
2025-11-06 23:19:27,699 - INFO - ✅ Data untuk kebun_jeruk: 744 records, columns: ['datetime', 'u_wind_10m', 'v_wind_10m', 'dewpoint_temperature_2m', 'temperature_2m', 'location_name', 'latitude', 'longitude', 'wind_speed_10m', 'wind_direction_10m', 'temperature_2m_c', 'dewpoint_temperature_2m_c']
2025-11-06 23:19:27,797 - INFO - ✅ File CSV disimpan: C:\Users\user\OneDrive\IPB\Thesis\02. Development\02. Data ERA5\01. Data Praprocessing\csv_outpu

...

       datetime  u_wind_10m  v_wind_10m  dewpoint_temperature_2m  temperature_2m  location_name  latitude   longitude  wind_speed_10m  wind_direction_10m  temperature_2m_c  dewpoint_temperature_2m_c
0    2023-10-01   -1.237019    0.699842               295.908295      299.254578    bundaran_hi    -6.206  106.903999        1.421265          150.501038         26.104584                  22.758301
1488 2023-10-01   -0.630329    0.990385               294.463593      296.853210      jagakarsa    -6.457  106.903999        1.173958          122.474670         23.703217                  21.313599
2976 2023-10-01   -0.911442    0.837797               295.552155      299.171753    kebun_jeruk    -6.206  106.653000        1.237994          137.410767         26.021759                  22.402161
744  2023-10-01   -1.237019    0.699842               295.908295      299.254578  kelapa_gading    -6.206  106.903999        1.421265          150.501038         26.104584                  22.758301
2232 

2025-11-06 23:19:29,508 - INFO - 📍 Memproses lokasi: lubang_buaya
2025-11-06 23:19:29,511 - INFO - Target: (-6.2889, 106.9092) -> Grid: (-6.2060, 106.9040)
2025-11-06 23:19:29,521 - INFO - Columns in DataFrame: ['valid_time', 'u10', 'v10', 'd2m', 't2m']
2025-11-06 23:19:29,531 - INFO - ✅ Data untuk lubang_buaya: 720 records, columns: ['datetime', 'u_wind_10m', 'v_wind_10m', 'dewpoint_temperature_2m', 'temperature_2m', 'location_name', 'latitude', 'longitude', 'wind_speed_10m', 'wind_direction_10m', 'temperature_2m_c', 'dewpoint_temperature_2m_c']
2025-11-06 23:19:29,532 - INFO - 📍 Memproses lokasi: kebun_jeruk
2025-11-06 23:19:29,534 - INFO - Target: (-6.2073, 106.7532) -> Grid: (-6.2060, 106.6530)
2025-11-06 23:19:29,544 - INFO - Columns in DataFrame: ['valid_time', 'u10', 'v10', 'd2m', 't2m']
2025-11-06 23:19:29,553 - INFO - ✅ Data untuk kebun_jeruk: 720 records, columns: ['datetime', 'u_wind_10m', 'v_wind_10m', 'dewpoint_temperature_2m', 'temperature_2m', 'location_name', 'latitude'

       datetime  u_wind_10m  v_wind_10m  dewpoint_temperature_2m  temperature_2m  location_name  latitude   longitude  wind_speed_10m  wind_direction_10m  temperature_2m_c  dewpoint_temperature_2m_c
0    2023-11-01   -0.248286    1.182060               297.310944      300.142761    bundaran_hi    -6.206  106.903999        1.207854          101.862244         26.992767                  24.160950
1440 2023-11-01    0.194707    0.941246               296.174866      298.400696      jagakarsa    -6.457  106.903999        0.961174           78.312561         25.250702                  23.024872
2880 2023-11-01   -0.310740    1.354133               296.770905      300.693665    kebun_jeruk    -6.206  106.653000        1.389330          102.924194         27.543671                  23.620911
720  2023-11-01   -0.248286    1.182060               297.310944      300.142761  kelapa_gading    -6.206  106.903999        1.207854          101.862244         26.992767                  24.160950
2160 

2025-11-06 23:19:29,853 - INFO - Columns in DataFrame: ['valid_time', 'u10', 'v10', 'd2m', 't2m']
2025-11-06 23:19:29,874 - INFO - ✅ Data untuk jagakarsa: 744 records, columns: ['datetime', 'u_wind_10m', 'v_wind_10m', 'dewpoint_temperature_2m', 'temperature_2m', 'location_name', 'latitude', 'longitude', 'wind_speed_10m', 'wind_direction_10m', 'temperature_2m_c', 'dewpoint_temperature_2m_c']
2025-11-06 23:19:29,877 - INFO - 📍 Memproses lokasi: lubang_buaya
2025-11-06 23:19:29,881 - INFO - Target: (-6.2889, 106.9092) -> Grid: (-6.2060, 106.9040)
2025-11-06 23:19:29,935 - INFO - Columns in DataFrame: ['valid_time', 'u10', 'v10', 'd2m', 't2m']
2025-11-06 23:19:29,975 - INFO - ✅ Data untuk lubang_buaya: 744 records, columns: ['datetime', 'u_wind_10m', 'v_wind_10m', 'dewpoint_temperature_2m', 'temperature_2m', 'location_name', 'latitude', 'longitude', 'wind_speed_10m', 'wind_direction_10m', 'temperature_2m_c', 'dewpoint_temperature_2m_c']
2025-11-06 23:19:29,977 - INFO - 📍 Memproses lokasi: 

       datetime  u_wind_10m  v_wind_10m  dewpoint_temperature_2m  temperature_2m  location_name  latitude   longitude  wind_speed_10m  wind_direction_10m  temperature_2m_c  dewpoint_temperature_2m_c
0    2023-12-01    1.230274   -0.545294               297.424927      298.932800    bundaran_hi    -6.206  106.903999        1.345704          336.095642         25.782806                  24.274933
1488 2023-12-01    0.622425    0.126825               296.474548      297.080444      jagakarsa    -6.457  106.903999        0.635214           11.516937         23.930450                  23.324554
2976 2023-12-01    1.084537   -0.156591               297.516083      298.894165    kebun_jeruk    -6.206  106.653000        1.095783          351.784088         25.744171                  24.366089
744  2023-12-01    1.230274   -0.545294               297.424927      298.932800  kelapa_gading    -6.206  106.903999        1.345704          336.095642         25.782806                  24.274933
2232 

In [5]:
import pandas as pd
import numpy as np
import os
import glob
import logging
from pathlib import Path
from datetime import datetime

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class CSVCombiner:
    """Class untuk menggabungkan semua file CSV menjadi satu file daily dan hourly"""
    
    def __init__(self, base_path):
        self.BASE_PATH = base_path
        self.CSV_OUTPUT_PATH = os.path.join(base_path, 'csv_output')
        self.COMBINED_OUTPUT_PATH = os.path.join(base_path, 'combined_output')
        
        # Buat direktori jika belum ada
        os.makedirs(self.COMBINED_OUTPUT_PATH, exist_ok=True)
        
        # Daftar variabel yang akan diaggregasi
        self.DAILY_AGGREGATIONS = {
            # Temperature
            'temperature_2m_c': ['mean', 'min', 'max'],
            'temperature_2m_k': ['mean', 'min', 'max'],
            
            # Dewpoint
            'dewpoint_temperature_2m_c': ['mean', 'min', 'max'],
            'dewpoint_temperature_2m_k': ['mean', 'min', 'max'],
            
            # Wind
            'wind_speed_10m': ['mean', 'max'],
            'u_wind_10m': ['mean'],
            'v_wind_10m': ['mean'],
            'wind_direction_10m': ['mean'],
            
            # Humidity
            'relative_humidity': ['mean', 'min', 'max'],
            
            # Precipitation
            'precipitation_mm': ['sum', 'max'],
            'total_precipitation': ['sum', 'max'],
            
            # Pressure
            'surface_pressure': ['mean', 'min', 'max'],
            'mean_sea_level_pressure': ['mean', 'min', 'max'],
        }
    
    def find_all_csv_files(self):
        """Mencari semua file CSV di folder csv_output"""
        pattern = os.path.join(self.CSV_OUTPUT_PATH, "*.csv")
        csv_files = glob.glob(pattern)
        
        # Juga cari di subfolder jika ada
        pattern_recursive = os.path.join(self.CSV_OUTPUT_PATH, "**", "*.csv")
        csv_files.extend(glob.glob(pattern_recursive, recursive=True))
        
        # Remove duplicates
        csv_files = sorted(list(set(csv_files)))
        
        logger.info(f"Menemukan {len(csv_files)} file CSV:")
        for i, file in enumerate(csv_files, 1):
            file_size = os.path.getsize(file) / 1024  # KB
            logger.info(f"  {i:2d}. {Path(file).name} ({file_size:.1f} KB)")
        
        return csv_files
    
    def read_and_validate_csv(self, csv_file):
        """Membaca dan memvalidasi file CSV"""
        try:
            logger.info(f"Membaca file: {Path(csv_file).name}")
            
            # Baca CSV file
            df = pd.read_csv(csv_file)
            
            # Validasi kolom minimum yang diperlukan
            required_cols = ['datetime', 'location_name', 'latitude', 'longitude']
            missing_cols = [col for col in required_cols if col not in df.columns]
            
            if missing_cols:
                logger.warning(f"File {Path(csv_file).name} missing columns: {missing_cols}")
                return None
            
            # Konversi kolom datetime
            df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
            
            # Hapus rows dengan datetime invalid
            original_count = len(df)
            df = df.dropna(subset=['datetime'])
            if len(df) < original_count:
                logger.warning(f"  Dihapus {original_count - len(df)} rows dengan datetime invalid")
            
            # Validasi data numerik
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            for col in numeric_cols:
                if col not in ['latitude', 'longitude']:
                    # Replace extreme outliers dengan NaN
                    q1 = df[col].quantile(0.01)
                    q3 = df[col].quantile(0.99)
                    iqr = q3 - q1
                    lower_bound = q1 - 1.5 * iqr
                    upper_bound = q3 + 1.5 * iqr
                    
                    outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
                    if outliers > 0:
                        logger.info(f"  {outliers} outliers ditemukan di {col}")
                        df[col] = df[col].clip(lower_bound, upper_bound)
            
            logger.info(f"  ✅ Berhasil membaca {len(df)} rows")
            return df
            
        except Exception as e:
            logger.error(f"❌ Gagal membaca {Path(csv_file).name}: {str(e)}")
            return None
    
    def combine_all_csv_files(self, csv_files):
        """Menggabungkan semua file CSV menjadi satu DataFrame"""
        all_dataframes = []
        
        for csv_file in csv_files:
            df = self.read_and_validate_csv(csv_file)
            if df is not None:
                # Tambahkan source file info
                df['source_file'] = Path(csv_file).name
                all_dataframes.append(df)
        
        if not all_dataframes:
            logger.error("Tidak ada data yang berhasil dibaca!")
            return None
        
        # Gabungkan semua dataframe
        combined_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
        
        # Urutkan data
        combined_df.sort_values(['datetime', 'location_name'], inplace=True)
        combined_df.reset_index(drop=True, inplace=True)
        
        logger.info(f"📊 Total data setelah digabung: {len(combined_df):,} rows")
        logger.info(f"📍 Lokasi: {combined_df['location_name'].unique().tolist()}")
        logger.info(f"📅 Rentang waktu: {combined_df['datetime'].min()} hingga {combined_df['datetime'].max()}")
        
        return combined_df
    
    def create_hourly_data(self, combined_df):
        """Membuat dataset hourly dari data yang sudah digabung"""
        logger.info("Membuat dataset hourly...")
        
        # Pilih kolom yang relevan untuk hourly data
        hourly_cols = ['datetime', 'location_name', 'latitude', 'longitude']
        
        # Tambahkan semua variabel meteorologi yang ada
        meteo_vars = [
            'temperature_2m_c', 'temperature_2m_k',
            'dewpoint_temperature_2m_c', 'dewpoint_temperature_2m_k',
            'relative_humidity',
            'u_wind_10m', 'v_wind_10m', 'wind_speed_10m', 'wind_direction_10m',
            'precipitation_mm', 'total_precipitation',
            'surface_pressure', 'mean_sea_level_pressure'
        ]
        
        for var in meteo_vars:
            if var in combined_df.columns:
                hourly_cols.append(var)
        
        hourly_df = combined_df[hourly_cols].copy()
        
        # Optimasi memory
        hourly_df = self.optimize_dataframe(hourly_df)
        
        logger.info(f"✅ Dataset hourly: {len(hourly_df):,} records")
        logger.info(f"📋 Kolom hourly: {list(hourly_df.columns)}")
        
        return hourly_df
    
    def create_daily_data(self, hourly_df):
        """Mengkonversi data hourly ke daily aggregates"""
        logger.info("Mengkonversi data hourly ke daily...")
        
        # Extract date untuk grouping
        hourly_df['date'] = hourly_df['datetime'].dt.date
        
        daily_data = []
        
        # Group by date dan location
        for (date, location_name), group in hourly_df.groupby(['date', 'location_name']):
            if len(group) < 18:  # Skip jika kurang dari 18 observasi per hari
                continue
                
            daily_record = {
                'date': date,
                'location_name': location_name,
                'latitude': group['latitude'].iloc[0],
                'longitude': group['longitude'].iloc[0],
                'hourly_observations_count': len(group)
            }
            
            # Aggregasi untuk setiap variabel
            for var_base, aggs in self.DAILY_AGGREGATIONS.items():
                # Cek variasi nama kolom
                possible_names = [var_base]
                if var_base.endswith('_c'):
                    possible_names.append(var_base.replace('_c', '_k'))
                
                for var_name in possible_names:
                    if var_name in group.columns:
                        for agg_func in aggs:
                            col_name = f"{var_base}_daily_{agg_func}"
                            
                            try:
                                if agg_func == 'mean':
                                    daily_record[col_name] = group[var_name].mean()
                                elif agg_func == 'min':
                                    daily_record[col_name] = group[var_name].min()
                                elif agg_func == 'max':
                                    daily_record[col_name] = group[var_name].max()
                                elif agg_func == 'sum':
                                    daily_record[col_name] = group[var_name].sum()
                            except:
                                daily_record[col_name] = np.nan
                        break
            
            # Hitung additional metrics
            self.calculate_additional_daily_metrics(daily_record, group)
            
            daily_data.append(daily_record)
        
        daily_df = pd.DataFrame(daily_data)
        
        # Urutkan data
        daily_df.sort_values(['date', 'location_name'], inplace=True)
        daily_df.reset_index(drop=True, inplace=True)
        
        # Optimasi memory
        daily_df = self.optimize_dataframe(daily_df)
        
        logger.info(f"✅ Dataset daily: {len(daily_df):,} records")
        logger.info(f"📋 Kolom daily: {list(daily_df.columns)}")
        
        return daily_df
    
    def calculate_additional_daily_metrics(self, daily_record, hourly_group):
        """Hitung metrik tambahan untuk data daily"""
        
        # Daily temperature range
        if 'temperature_2m_c_daily_max' in daily_record and 'temperature_2m_c_daily_min' in daily_record:
            daily_record['temperature_2m_c_daily_range'] = (
                daily_record['temperature_2m_c_daily_max'] - daily_record['temperature_2m_c_daily_min']
            )
        
        # Rainy day flag
        if 'precipitation_mm_daily_sum' in daily_record:
            daily_record['is_rainy_day'] = daily_record['precipitation_mm_daily_sum'] > 0.1
        
        # Wind dominance
        if 'u_wind_10m_daily_mean' in daily_record and 'v_wind_10m_daily_mean' in daily_record:
            u_mean = daily_record['u_wind_10m_daily_mean']
            v_mean = daily_record['v_wind_10m_daily_mean']
            daily_record['resultant_wind_speed'] = np.sqrt(u_mean**2 + v_mean**2)
    
    def optimize_dataframe(self, df):
        """Optimasi tipe data untuk menghemat memory"""
        
        # Optimasi numeric columns
        float_cols = df.select_dtypes(include=['float64']).columns
        for col in float_cols:
            if col in df.columns:
                # Cek range data untuk memilih tipe yang tepat
                if df[col].notna().any():
                    col_min = df[col].min()
                    col_max = df[col].max()
                    
                    if col_min >= -32768 and col_max <= 32767:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float32)
        
        # Optimasi string columns
        string_cols = df.select_dtypes(include=['object']).columns
        for col in string_cols:
            if col in df.columns:
                df[col] = df[col].astype('category')
        
        # Convert date columns
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
        
        return df
    
    def save_combined_data(self, hourly_df, daily_df):
        """Menyimpan data combined ke file CSV"""
        try:
            # File paths
            hourly_output = os.path.join(self.COMBINED_OUTPUT_PATH, "era5_combined_hourly.csv")
            daily_output = os.path.join(self.COMBINED_OUTPUT_PATH, "era5_combined_daily.csv")
            
            # Simpan hourly data
            hourly_df.to_csv(hourly_output, index=False)
            logger.info(f"✅ Data hourly disimpan: {hourly_output}")
            
            # Simpan daily data
            daily_df.to_csv(daily_output, index=False)
            logger.info(f"✅ Data daily disimpan: {daily_output}")
            
            # Simpan juga sebagai Parquet untuk efisiensi
            try:
                hourly_parquet = os.path.join(self.COMBINED_OUTPUT_PATH, "era5_combined_hourly.parquet")
                daily_parquet = os.path.join(self.COMBINED_OUTPUT_PATH, "era5_combined_daily.parquet")
                
                hourly_df.to_parquet(hourly_parquet, index=False)
                daily_df.to_parquet(daily_parquet, index=False)
                
                logger.info(f"✅ Data Parquet disimpan: {hourly_parquet}, {daily_parquet}")
            except:
                logger.warning("Tidak bisa menyimpan format Parquet")
            
            return hourly_output, daily_output
            
        except Exception as e:
            logger.error(f"❌ Error saving combined data: {str(e)}")
            return None, None
    
    def generate_summary_report(self, hourly_df, daily_df):
        """Generate summary report lengkap"""
        logger.info("\n" + "="*60)
        logger.info("📊 COMBINED DATA SUMMARY REPORT")
        logger.info("="*60)
        
        # Hourly data summary
        logger.info("HOURLY DATA:")
        logger.info(f"  • Total records: {len(hourly_df):,}")
        logger.info(f"  • Date range: {hourly_df['datetime'].min()} to {hourly_df['datetime'].max()}")
        logger.info(f"  • Locations: {', '.join(hourly_df['location_name'].unique())}")
        logger.info(f"  • Variables available: {len([col for col in hourly_df.columns if col not in ['datetime', 'location_name', 'latitude', 'longitude', 'source_file']])}")
        
        # Daily data summary
        logger.info("\nDAILY DATA:")
        logger.info(f"  • Total records: {len(daily_df):,}")
        logger.info(f"  • Date range: {daily_df['date'].min()} to {daily_df['date'].max()}")
        logger.info(f"  • Locations: {', '.join(daily_df['location_name'].unique())}")
        logger.info(f"  • Average observations per day: {daily_df['hourly_observations_count'].mean():.1f}")
        
        # Data quality metrics
        logger.info("\n📈 DATA QUALITY METRICS:")
        
        # Hourly data completeness
        hourly_vars = [col for col in hourly_df.columns if col not in ['datetime', 'date', 'location_name', 'latitude', 'longitude', 'source_file']]
        for var in hourly_vars[:6]:  # Tampilkan 6 variabel pertama
            completeness = hourly_df[var].notna().mean() * 100
            logger.info(f"  • {var}: {completeness:.1f}% complete")
        
        # Daily data statistics
        logger.info("\n📊 DAILY STATISTICS (Sample):")
        temp_stats = [
            f"Avg: {daily_df['temperature_2m_c_daily_mean'].mean():.1f}°C",
            f"Min: {daily_df['temperature_2m_c_daily_min'].min():.1f}°C", 
            f"Max: {daily_df['temperature_2m_c_daily_max'].max():.1f}°C"
        ]
        logger.info(f"  • Temperature: {', '.join(temp_stats)}")
        
        if 'precipitation_mm_daily_sum' in daily_df.columns:
            rain_days = (daily_df['precipitation_mm_daily_sum'] > 0.1).sum()
            logger.info(f"  • Rainy days: {rain_days}/{len(daily_df)} ({rain_days/len(daily_df)*100:.1f}%)")
        
        # Data preview
        logger.info(f"\n👀 DAILY DATA PREVIEW (first 5 rows):")
        preview_cols = ['date', 'location_name', 'temperature_2m_c_daily_mean', 
                       'relative_humidity_daily_mean', 'precipitation_mm_daily_sum']
        available_preview_cols = [col for col in preview_cols if col in daily_df.columns]
        print(daily_df[available_preview_cols].head().to_string(index=False))
    
    def run_combination(self):
        """Menjalankan proses penggabungan seluruhnya"""
        logger.info("🚀 MEMULAI PROCESS GABUNGKAN SEMUA CSV FILES")
        logger.info("=" * 60)
        
        # Step 1: Cari semua file CSV
        csv_files = self.find_all_csv_files()
        if not csv_files:
            logger.error("Tidak ada file CSV yang ditemukan!")
            return
        
        # Step 2: Baca dan gabungkan semua CSV
        combined_df = self.combine_all_csv_files(csv_files)
        if combined_df is None:
            return
        
        # Step 3: Buat dataset hourly
        hourly_df = self.create_hourly_data(combined_df)
        
        # Step 4: Buat dataset daily
        daily_df = self.create_daily_data(hourly_df)
        
        # Step 5: Simpan data
        hourly_file, daily_file = self.save_combined_data(hourly_df, daily_df)
        
        # Step 6: Generate report
        self.generate_summary_report(hourly_df, daily_df)
        
        logger.info(f"\n🎉 PROCESS SELESAI!")
        logger.info(f"📁 File output tersimpan di: {self.COMBINED_OUTPUT_PATH}")
        logger.info(f"📄 File hourly: {hourly_file}")
        logger.info(f"📄 File daily: {daily_file}")

def main():
    """Main function"""
    base_path = 'C:\\Users\\user\\OneDrive\\IPB\\Thesis\\02. Development\\02. Data ERA5\\01. Data Praprocessing'
    
    combiner = CSVCombiner(base_path)
    combiner.run_combination()

if __name__ == "__main__":
    main()

2025-11-06 23:19:30,444 - INFO - 🚀 MEMULAI PROCESS GABUNGKAN SEMUA CSV FILES
2025-11-06 23:19:30,447 - INFO - ============================================================
2025-11-06 23:19:30,459 - INFO - Menemukan 10 file CSV:
2025-11-06 23:19:30,463 - INFO -    1. data_era5_hourly_2023_01_per_location.csv (462.1 KB)
2025-11-06 23:19:30,465 - INFO -    2. data_era5_hourly_2023_02_per_location.csv (417.2 KB)
2025-11-06 23:19:30,467 - INFO -    3. data_era5_hourly_2023_05_per_location.csv (467.5 KB)
2025-11-06 23:19:30,471 - INFO -    4. data_era5_hourly_2023_06_per_location.csv (452.4 KB)
2025-11-06 23:19:30,474 - INFO -    5. data_era5_hourly_2023_07_per_location.csv (467.1 KB)
2025-11-06 23:19:30,477 - INFO -    6. data_era5_hourly_2023_08_per_location.csv (467.4 KB)
2025-11-06 23:19:30,479 - INFO -    7. data_era5_hourly_2023_09_per_location.csv (452.0 KB)
2025-11-06 23:19:30,481 - INFO -    8. data_era5_hourly_2023_10_per_location.csv (467.4 KB)
2025-11-06 23:19:30,483 - INFO -    9

TypeError: Categorical is not ordered for operation min
you can use .as_ordered() to change the Categorical to an ordered one
